In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from datasets.lyft import *
from datasets.carla import compile_data as cpdc
from tools.utils import *
from tools.loss import *

%matplotlib inline
import matplotlib.pyplot as plt

from numpy.linalg import inv

In [ ]:
from datasets.nuscenes import compile_data as  cpdn


In [ ]:
loader = compile_data("ood", "trainval", "../../data/lyft", "vehicle")

In [ ]:
len(loader.dataset)

In [ ]:
loader_n = cpdn("val", "mini", "../../data/nuscenes", "vehicle")

In [ ]:
model = Evidential(backbone="lss", devices=[7])
# model.load(torch.load("../outputs/aug/19.pt"))

In [ ]:
images, intrinsics, extrinsics, labels, oods = loader.dataset[1]
images = images[None].to(7)
intrinsics = intrinsics[None].to(7)
extrinsics = extrinsics[None].to(7)
pts = model.backbone.module.get_geometry(intrinsics, extrinsics).cpu().numpy()

In [ ]:
oods.sum()

In [ ]:
plt.imshow(1-oods[0].numpy())
plt.axis("off")

In [ ]:
# plt.imshow(map_rgb(labels, ego=True) / 255)

cam = ['fl', 'f', 'fr', 'bl', 'b', 'br']

for i in range(6):
    plt.scatter(pts[:, i, :, :, :, 0].reshape(-1) * 2 + 100, pts[:, i, :, :, :, 1].reshape(-1) * 2+ 100, s=15, label=cam[i], alpha=.2, edgecolors='face')

plt.xlim(0, 250)
# plt.ylim(0, 200)
plt.legend(loc='upper right')